<a href="https://colab.research.google.com/github/passivebook/FixWordPressLinks/blob/main/Fix_WordPress_Links.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://drive.google.com/drive/folders/1T5JJmlDwdaEkAAbpH2VOn-TCeAuW8OKl

In [13]:
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#### Put the path to the google drive folder after `/content/gdrive/My Drive/`



In [14]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Fix WordPress Links")

#### Upload the post.txt file to Google Drive 
1. Create a `post.txt` file and paste the WordPress Post Code in it. 
2. Upload the post.txt file to path setup above. 

### Check file post.txt exists in Drive by running the code below

In [15]:
! ls

'Fix WordPress Links.ipynb'   input.txt   output.txt


### File Name Uploaded

In [16]:
filename = 'input.txt'
domain = 'passivebook.com'
affiliate_link = 'passivebook.com/go'

In [17]:
html_doc = """
<!-- wp:paragraph -->
<p><a class="test" href="https://passivebook.com/go/trello"> Affiliate Link</a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="https://passivebook.com/calculators/blog-earning-calculator/" class="test">Internal Link </a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="https://google.com" rel="nofollow">External Link</a></p>
<!-- /wp:paragraph -->
"""

soup = BeautifulSoup(html_doc, 'html.parser')

In [ ]:
f = open(filename)
content = f.read()
#parse HTML
soup = BeautifulSoup(content, 'html.parser')

In [19]:
# Operations on All Links
for link in soup.find_all('a'):
    # Set All Links to Open in New Tab
    link['target'] = '_blank'
    # Remove Leading and Trailiing White Spaces
    link.string = link.string.strip()

# Internal Links (Includes Affiliate Links)
for link in soup.find_all('a', href=re.compile(domain)):
    url = link['href']
    if url[len(url)-1] != "/":
        link['href'] = url + "/"

# Affiliate Links
for link in soup.find_all('a', href=re.compile(affiliate_link)):
    link['rel'] = 'nofollow'

# External Links
#for link in soup.find_all('a', href=re.compile(".*?:\\/\\/(?!"+domain+").*?")):
#    link['rel'] = link['rel'].remove('nofollow')
#    link['rel'] = 'nofollow'

# print(soup)

In [ ]:
with open("output.txt", "w", encoding='utf-8') as file:
    file.write(str(soup))